In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
import warnings
warnings.filterwarnings('ignore')

from ckonlpy.tag import Twitter; t= Twitter()
from ckonlpy.tag import Postprocessor
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 데이터 불러오기

In [2]:
olive_man_skin = pd.read_csv('../dataset/oliveyoung_product_info.csv')
rew = pd.read_csv('../dataset/glowpick_reviews.csv')
#g_prod = pd.read_csv('../dataset/glowpick_products.csv')

# 데이터 확인

In [3]:
print('shape:',rew.shape)
print('\n#### 결측치 갯수 ####\n')
print(rew.isnull().sum())
rew.head()

shape: (5125, 7)

#### 결측치 갯수 ####

date             0
user_id          0
sex              0
age_skin_type    0
rate             0
content          1
product_url      0
dtype: int64


,date,user_id,sex,age_skin_type,rate,content,product_url
0,28일 전,minseo6666,f,16세 · 복합성 ·,good,아빠 사드렸는데 저아해여,/product/7212
1,2개월 전,피부에관심있는남자,m,20세 · 복합성 ·,good,향이 좋아요 근데 좀 제 피부가 민감해서,/product/7212
2,3개월 전,타라타라,f,23세 · 복합성 ·,good,가격이 저렴하고 냄새가 안 쎄서 좋아요. 👍,/product/7212
3,4개월 전,흐시너,f,18세 · 복합성 ·,good,선물용으로 사서 선물했는데 나름 만족했는지 매일 바른다네요;,/product/7212
4,4개월 전,홍끼루,f,32세 · 복합성 ·,good,향이 청량해서 좋아요.,/product/7212


- 나이와 피부타입이 합쳐져있다. 따로 분리가 필요하다.
- content 속성에 결측치가 있다 처리가 필요하다.

# 전처리

In [4]:
np.array([i.split('·') for i in rew['age_skin_type']])[:,0]
rew['age']= np.array([i.split('·') for i in rew['age_skin_type']])[:,0]
rew['skin_type']= np.array([i.split('·') for i in rew['age_skin_type']])[:,1]

In [5]:
del rew['age_skin_type']

In [6]:
rew.loc[rew['content'].isnull()]

,date,user_id,sex,rate,content,product_url,age,skin_type
2322,2016.12.4,En960,f,good,NaN,/product/82765,24세,건성


- 결측치가 하나 보인다. 결측치가 하나뿐이니 이를 제거해준다.

In [7]:
rew = rew.dropna().copy()

In [8]:
rew.head(2)

,date,user_id,sex,rate,content,product_url,age,skin_type
0,28일 전,minseo6666,f,good,아빠 사드렸는데 저아해여,/product/7212,16세,복합성
1,2개월 전,피부에관심있는남자,m,good,향이 좋아요 근데 좀 제 피부가 민감해서,/product/7212,20세,복합성


# CKonlpy

https://github.com/lovit/customized_konlpy

- KoNLPy의 customized version이다.
- meceb, khaiii는 윈도위 버전을 지원하지 않아 제외하였다.
- 사용자 사전을 쉽게 사용할 수 있다.
- 단어 치환을 위한 후처리를 할 수 있다.
    - passwords 에 등록된 단어, (단어, 품사)만 출력됩니다.
    - stopwords 에 등록된 단어, (단어, 품사)는 출력되지 않습니다.
    - 특정 품사를 지정하면, 해당 품사만 출력됩니다.
    - 치환할 단어, (단어, 품사)를 dict 형식으로 정의하면 tag 에서 단어가 치환되어 출력됩니다.

# 리뷰 전처리

In [9]:
print('리뷰 총 갯수: {}'.format(rew['content'].shape[0]))

리뷰 총 갯수: 5124


In [10]:
def preprocessing(text):
    # 개행문자 제거(줄바꿈 문자)
    text = re.sub('\n', ' ', text)
    # 의성어 제거
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ!?~]', ' ',text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기서는 제거
    # re.sub : 첫번째 인수에 바꿀 문자열을 입력하고 두 번째 인수에 대상 문자열을 입력한다.
    # text = re.sub('[?.,;:|\)*~'!^\- +<>@\#$%&-=#}*]','',text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]',' ',text)
    # [^5]와 같이 '^'가 []안에서 쓰이는 경우에는 5를 제외한 모든 문자를 나타냄
    # 가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z을 만족하는 것들을 제외한 것들을 ' '로 치환
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub(' +',' ',text)
    return text

In [11]:
rew['content'] = rew['content'].map(str)

In [12]:
rew['rew_pre'] = rew['content'].apply(preprocessing)

In [13]:
rew['rew_pre'].head()

0                        아빠 사드렸는데 저아해여
1               향이 좋아요 근데 좀 제 피부가 민감해서
2               가격이 저렴하고 냄새가 안 쎄서 좋아요 
3    선물용으로 사서 선물했는데 나름 만족했는지 매일 바른다네요 
4                         향이 청량해서 좋아요 
Name: rew_pre, dtype: object

In [14]:
rew['morphs'] = rew['rew_pre'].map(t.morphs)
#rew['morphs'] = rew['morphs'].map(' '.join)

In [15]:
rew['rate'].value_counts()

good     2214
best     1476
soso      979
bad       360
worst      95
Name: rate, dtype: int64

In [16]:
rew_r = rew['rew_pre']

In [17]:
worst_r = rew.loc[rew['rate'] == 'worst']['rew_pre']
bad_r = rew.loc[rew['rate'] == 'bad']['rew_pre']
soso_r = rew.loc[rew['rate'] == 'soso']['rew_pre']
best_r = rew.loc[rew['rate'] == 'best']['rew_pre']
good_r = rew.loc[rew['rate'] == 'good']['rew_pre']
worst_r_a = ' '.join(worst_r)
bad_r_a = ' '.join(bad_r)
soso_r_a = ' '.join(soso_r)
best_r_a = ' '.join(best_r)
good_r_a = ' '.join(good_r)
rew_r_a = ' '.join(rew_r)

# 불용어 정의

In [18]:
rew_n = Postprocessor(t,passtags = 'Noun')
rew_v = Postprocessor(t,passtags = 'Verb')
rew_a = Postprocessor(t,passtags = 'Adjective')
rew_j = Postprocessor(t,passtags = 'Josa')

In [19]:
#rew['token'] = [i.split(' ') for i in rew['rew_pre']]

In [21]:
rew_n_df = pd.DataFrame(rew_n.pos(rew_r_a)) # 명사
rew_v_df = pd.DataFrame(rew_v.pos(rew_r_a)) # 동사
rew_a_df = pd.DataFrame(rew_a.pos(rew_r_a)) # 형용사
rew_j_df = pd.DataFrame(rew_j.pos(rew_r_a)) # 조사

In [22]:
stop_words = ['거','어요','단','것','평가','때','이','리뷰','어서','좀','저','제','요','더','고','진짜','쓰기','생각','입니다','같아요','있','같','써','쓰고','들']

# replace

In [23]:
replace = {'좋아요':'좋다','좋':'좋다','좋은':'좋다','좋고':'좋다','좋다고':'좋다','좋은데':'좋다','좋습니다':'좋다','좋아서':'좋다','좋았':'좋다','좋은데':'좋다','좋네요':'좋다','새빨간':'빨간','약해서':'약하다','남친':'남자친구','남자':'남성'}

In [24]:
postprocessor_r = Postprocessor(t, replace = replace) # replace

In [25]:
# 평점 등급별 리뷰 확인
worst_r = rew.loc[rew['rate'] == 'worst']['rew_pre']
bad_r = rew.loc[rew['rate'] == 'bad']['rew_pre']
soso_r = rew.loc[rew['rate'] == 'soso']['rew_pre']
best_r = rew.loc[rew['rate'] == 'best']['rew_pre']
good_r = rew.loc[rew['rate'] == 'good']['rew_pre']
worst_r_a = ' '.join(worst_r)
bad_r_a = ' '.join(bad_r)
soso_r_a = ' '.join(soso_r)
best_r_a = ' '.join(best_r)
good_r_a = ' '.join(good_r)
rew_r_a = ' '.join(rew_r)

In [26]:
worst_re = np.array(postprocessor_r.pos(worst_r_a))[:,0]
bad_re = np.array(postprocessor_r.pos(bad_r_a))[:,0]
soso_re = np.array(postprocessor_r.pos(soso_r_a))[:,0]
best_re = np.array(postprocessor_r.pos(best_r_a))[:,0]
good_re = np.array(postprocessor_r.pos(good_r_a))[:,0]

# 불용어 제거

In [27]:
# 불용어 추가한 것들을 제거해준다.
result_worst = []; result_bed = []; result_soso = []; result_best = []; result_good = []

[result_worst.append(i) for i in worst_re if i not in stop_words]
[result_bed.append(i) for i in worst_re if i not in stop_words]
[result_soso.append(i) for i in worst_re if i not in stop_words]
[result_best.append(i) for i in worst_re if i not in stop_words]
[result_good.append(i) for i in worst_re if i not in stop_words]

print(result_worst[:20])

['비오템', '입', '문하', '자마자', '떠나게', '한', '제품', '피부', '에', '신경안', '쓰던', '적', '에', '올인원', '제품', '하나', '만', '겨울', '에', '바를']


In [28]:
rew.loc[rew['rate'] == 'worst'][['rew_pre','morphs']].reset_index().iloc[30:35,]

,index,rew_pre,morphs
30,3130,이거 쓰고 피부 다 뒤집어졌던 기억이 나네요 단순히 여드름이나 트러블이 났던게 아니...,"[이, 거, 쓰고, 피부, 다, 뒤집어졌던, 기억, 이, 나네요, 단순, 히, 여드..."
31,3204,이건 걍 돈아까음 얼굴 넘 건조해서 뱅기타기전에 면세에서 샀는데 걍 다른거랑 다를게...,"[이건, 걍, 돈, 아까음, 얼굴, 넘, 건조, 해서, 뱅, 기타, 기전, 에, 면..."
32,3222,그냥 물임,"[그냥, 물임]"
33,3269,향이 남자화장품냄새 딱 그건데,"[향, 이, 남자, 화장품, 냄새, 딱, 그건, 데]"
34,3615,거짓된 양 내구성 부족 제품을 만족시키지 못하는 부담스러운 가격,"[거짓, 된, 양, 내, 구성, 부족, 제품, 을, 만족, 시키지, 못, 하는, 부..."


In [29]:
t.pos('시원함') # 원함까지함

[('시원', 'Noun'), ('함', 'Noun')]

In [30]:
print(result_worst[:20])

['비오템', '입', '문하', '자마자', '떠나게', '한', '제품', '피부', '에', '신경안', '쓰던', '적', '에', '올인원', '제품', '하나', '만', '겨울', '에', '바를']


# 띄어쓰기 수정

In [33]:
def find_word_idx(word):
    cnt = 0
    cnt_l1 = []
    for i in rew['morphs']:
        if word in i:
            cnt_l1.append(cnt)
        cnt += 1
    return cnt_l1

In [34]:
rew['rew_pre'][132] = '비오템 입문 하자마자 떠나게 한 제품 피부에 신경안쓰던적에 올인원 제품 하나만 겨울에 바를만큼 무신경했는데 요 라인은 참을수 없이 건조했다 '
rew['rew_pre'][2435] = '최악중최악 바르면 얼굴따갑고 아침엔트러블로가득찬 내얼굴 한번쯤 여드름쟁이가 되고 싶다면 추천 하는 크림'
rew['rew_pre'][2818]= '내가 흑우인지 괜히 면도세안 한꺼번에 되는 폼이라길래 구매했다가 제대로 돈날린 제품 쫀쫀이고 쫀득이고 간에 일단 샤워하면서는 절대 쓸수 없었다 어찌된 영문인지 물에 상당히 약해서 얼굴이 세면후 상태로 촉촉하면 죄다 녹아 내려서 폼이 오일로 바뀌는 매직을 겪게된다 샤워 전 최저한의 수분으로 거품유지하며 씻는 스트레스에 매번 조절 실패해서 제품낭비 돈낭비 제대로 한 역대급 최악 클랜징 제품 지금은 차세안 하기위해서 그냥 세안 전 한번 사용해주고 전용 클랜징폼은 따로 쓴다 오히려 면도크림 용으로만 나왔었어도 평균은 가지 않았을까 하는 느낌 상당히 화한 느낌으로 트러블 없이 면도가 가능하긴 한데 이걸 면도크림만으로 쓰기엔 가격면에서 상당히 그냥 따로 면도할렵니다 '
rew['rew_pre'][2822] = '내 인생 최고의 똥템'
rew['rew_pre'][147] = '원래 sk 를 사용하다가 갈아탔는데 순한거같기는 하나 그렇게 피부가 좋아지는데 도움을 주는거는 크게 못느꼈어요 무난한 사용감 이긴해요'
# 무난
rew['rew_pre'].iloc[find_word_idx('난하')] = rew['rew_pre'].iloc[find_word_idx('난하')].map(lambda x: str(x).replace('무난하게','무난 하게'))

In [ ]:
#print('내', '가', '흑우', '인지', '괜히', '면도', '세안', '한꺼', '번', '에', '되는', '폼', '이라', '길래', '구매', '했', '다가', '제대로', '돈', '날린', '제품', '쫀쫀', '이고', '쫀득', '이고', '간', '에', '일단', '샤워', '하', '면서', '는', '절대', '쓸수', '없었다', '어찌', '된', '영문', '인지', '물', '에', '상당', '히', '약', '해서', '얼굴', '세면', '후', '상태', '로', '촉촉하면', '죄다', '녹', '아내', '려', '서', '폼', '오일', '로', '바뀌는', '매직', '을', '겪게', '된다', '샤워', '전', '최저', '한', '의', '수분', '으로', '거품', '유지', '하며', '씻는', '스트레스', '에', '매번', '조절', '실패', '해서', '제품', '낭비', '돈', '낭비', '제대로', '한', '역대', '급', '최악', '클랜', '징', '제품', '지금', '은', '차', '세안', '하기', '위해', '서', '그냥', '세안', '전', '한번', '사용', '해주', '전용', '클랜', '징폼', '은', '따로', '쓴다', '오히려', '면도', '크림', '용으', '로만', '나왔었', '어도', '평균', '은', '가지', '않았을까', '하는', '느낌', '상당', '히', '화한', '느낌', '으로', '트러블', '없이', '면도', '가', '가능', '하긴', '한데', '걸', '면도', '크림', '만으로', '엔', '가격', '면', '에서', '상당', '히', '그냥', '따로', '면도', '할렵니', '다')

In [35]:
rew['rew_pre'][3130]

'이거 쓰고 피부 다 뒤집어졌던 기억이 나네요 단순히 여드름이나 트러블이 났던게 아니라 턱쪽 전체에 피부염 같은게 났더래서 고생 좀 했었습니다 지금 생각해보니 바를 때 시원함을 넘어서 화한 기분이 들던게 느낌부터 싸했던거 같군요 매우 비추입니다 가격도 ml에 원이라니 차라리 그 돈으로 우유 ml 개 사서 얼굴에 부비겠습니다 길바닥 개미들에게 사탕이나 젤리 사서 나눠주는게 차라리 돈 덜 아까울 것 같습니다 원 주고 이거 써서 피부병 걸린 후에 피부과 다니느라 만원도 더 썼었네요 아무튼 매우 비추 '

# 사전추가

In [36]:
t.add_dictionary('잔고통', 'Noun')
t.add_dictionary('유분기', 'Noun')
t.add_dictionary('남성미', 'Noun')
t.add_dictionary('한꺼번에','Adverb')
t.add_dictionary('클랜징', 'Noun')
t.add_dictionary('쉐이빙폼', 'Noun')
t.add_dictionary('시원함', 'Noun')
t.add_dictionary('끈적', 'Noun')
t.add_dictionary('유분', 'Noun')
t.add_dictionary('인생템', 'Noun')
t.add_dictionary('전체적', 'Noun')
t.add_dictionary('번들', 'Noun')
t.add_dictionary('사용감', 'Noun')
# 화장품을 사용목적이나 용도에 맞게 형태를 만드는것을 제형
t.add_dictionary('제형', 'Noun')
t.add_dictionary('세일', 'Noun')
t.add_dictionary('세정', 'Noun')
t.add_dictionary('보들', 'Noun')

- 명사, 명사로 나눠지는 경우 [ex) 전체적 > 전(명사) 체적(명사)] 그 단어를 명사로 추가해줘야 단어가 추가됨

In [37]:
t.template_tagger.templates

[('Adjective',),
 ('Adverb',),
 ('Conjunction',),
 ('Exclamation',),
 ('KoreanParticle',),
 ('Noun',),
 ('Verb',),
 ('Noun', 'Noun'),
 ('Noun', 'Josa'),
 ('Noun', 'Adjective'),
 ('Noun', 'Verb'),
 ('Modifier', 'Noun'),
 ('Noun', 'Noun', 'Adjective'),
 ('Noun', 'Noun', 'Josa'),
 ('Noun', 'Noun', 'Verb'),
 ('Modifier', 'Noun', 'Josa'),
 ('Josa',)]

# 추가 전처리(~ing)

In [38]:
rew_n_df.loc[:,0].value_counts()

제품        1113
향         1021
거          870
어요         813
피부         796
것          660
사용         646
선물         610
단          598
평가         576
남자         541
느낌         486
남자친구       479
때          476
리뷰         441
냄새         431
이          393
아빠         380
어서         361
남친         338
좀          323
저          309
가격         291
남성         285
스킨         282
제          276
그냥         266
요          258
얼굴         251
더          235
진짜         232
고          231
끈          231
로션         228
올인원        220
화장품        217
쓰기         214
데          206
머리         205
생각         203
크림         195
안          193
구매         192
분          192
발라         191
정말         190
다른         189
그          188
남편         188
다가         186
정도         184
용          178
후          178
하나         178
무난         176
추천         176
트러블        175
중          175
아하         174
함          174
듯          172
별로         169
지성         165
면서         164
수          163
일단         163
건조        

In [39]:
stop_words = ['거','어요','단','것','평가','때','이','리뷰','어서','좀','저','제','요','더','고','진짜','쓰기','생각','입니다','같아요','있','같','써','쓰고','들','그냥','데','안','그','다가','정도','후','용','하나','중','아하','함','듯','면서','수','일단','한번','걸','편','사','뭐','형','유','나','이나','보고','이건','내','까지','성','아주','사드','젤','적','쓰기','때']

In [40]:
rew['rew_n'] = rew['rew_pre'].apply(rew_n.pos)

In [41]:
rew['rew_pre'][0]

'아빠 사드렸는데 저아해여'

In [42]:
rew['rew_n'][0]

[('아빠', 'Noun'),
 ('사드', 'Noun'),
 ('렸는데', 'Noun'),
 ('저', 'Noun'),
 ('아해', 'Noun')]

In [ ]:
#rew['morphs']

In [43]:
'향' in rew['morphs'][1]

True

In [44]:
def find_word_idx(word):
    cnt = 0
    cnt_l1 = []
    for i in rew['morphs']:
        if word in i:
            cnt_l1.append(cnt)
        cnt += 1
    return cnt_l1

In [45]:
rew['rew_pre'].iloc[find_word_idx('자')].iloc[19]

'일단 향이별로입니다 요즘그다지 선호하지않을 조금 자극적인향입니다 그리고 보습력도 그다지 '

In [ ]:
#rew['rew_pre'][147] = '원래 sk 를 사용하다가 갈아탔는데 순한거같기는 하나 그렇게 피부가 좋아지는데 도움을 주는거는 크게 못느꼈어요 무난한 사용감 이긴해요'

In [ ]:
#rew['rew_pre'].iloc[find_word_idx('감')].iloc[15]

In [46]:
t.pos('자극적인향입니다')

[('자', 'Noun'),
 ('극적', 'Noun'),
 ('인', 'Josa'),
 ('향', 'Noun'),
 ('입니다', 'Adjective')]

In [ ]:
#t.add_dictionary('그래서', 'Con',force=True)

In [ ]:
#stop_words = ['거','어요','단','것','평가','때','이','리뷰','어서','좀','저','제','요','더','고','진짜','쓰기','생각','입니다','같아요','있','같','써','쓰고','들','그냥','데','그','다가','정도','후','용','하나','중','아하','함','듯','면서','수','일단','한번','걸','편','사','뭐','형','유','나','이나','보고','이건','내','까지','성','아주','사드','젤','적','쓰기','분','어서','편','뭐','나','이나','보고','이건','내','까지','성','임']

In [ ]:
#############33

# 띄어쓰기 모델

In [50]:
from soyspacing.countbase import CountSpace

In [51]:
model_fname = 'C:/Users/jhlee/OneDrive/home/processed/processed/space-correct.model'

In [52]:
model = CountSpace()

In [53]:
model.load_model(model_fname, json_format = False)

In [54]:
model.correct('일단 향이별로입니다 요즘그다지 선호하지않을 조금 자극적인향입니다 그리고 보습력도 그다지 ')

('일단 향이별로입니다 요즘그다지 선호하지 않을 조금 자극적인향입니다 그리고 보습력도 그다지',
 [None,
  1,
  None,
  None,
  0,
  None,
  None,
  0,
  1,
  None,
  None,
  None,
  0,
  1,
  None,
  None,
  None,
  1,
  0,
  1,
  0,
  1,
  None,
  0,
  0,
  None,
  None,
  None,
  0,
  1,
  0,
  0,
  1,
  None,
  None,
  None,
  1,
  0,
  0,
  1])